# Getting Started with DCH

This notebook demonstrates a minimal, deterministic, and torch-free-by-default workflow for DCH. It covers CLI usage, environment gating for optional dependencies (torch/norse/tonic/bindsnet), and a tiny in-memory pipeline example. For full usage docs see [docs/USAGE.md](docs/USAGE.md:1). Optional dataset downloads are opt-in via [scripts/download_datasets.py](scripts/download_datasets.py:1).

In [ ]:
# Environment probe: detect optional dependencies without importing heavy submodules.
import importlib
import importlib.util as ilu

def _have(mod: str) -> bool:
    try:
        return ilu.find_spec(mod) is not None
    except Exception:
        return False

mods = ["torch", "norse", "tonic", "bindsnet"]
HAVE = {m: _have(m) for m in mods}
VERS = {}
for m, present in HAVE.items():
    if present:
        try:
            mod = importlib.import_module(m)
            ver = getattr(mod, "__version__", "unknown")
        except Exception:
            ver = "unknown"
        VERS[m] = ver

print("Optional dependencies status:")
for m in mods:
    if HAVE[m]:
        print(f" - {m}: {VERS.get(m, 'unknown')}")
    else:
        hint = (
            "pip install 'torch>=2.2'" if m == "torch" else
            "pip install 'norse>=0.0.9'" if m == "norse" else
            "pip install 'tonic>=1.4.0'" if m == "tonic" else
            "pip install 'bindsnet>=0.3'"
        )
        conda = ("conda install -c conda-forge pytorch" if m == "torch" else f"conda install -c conda-forge {m}")
        print(f" - {m}: not installed. Try: {hint}  | Or: {conda}")

HAVE_TORCH = HAVE.get("torch", False)
HAVE_NORSE = HAVE.get("norse", False)
HAVE_TONIC = HAVE.get("tonic", False)
HAVE_BINDSNET = HAVE.get("bindsnet", False)
if not HAVE_TORCH:
    print("Proceeding in torch-free mode by default.")

In [ ]:
# Determinism setup (seed stdlib, numpy if available, and torch if present)
import random
random.seed(0)
try:
    import numpy as _np
    _np.random.seed(0)
    print("NumPy seeded.")
except Exception:
    print("NumPy not available; skipping NumPy seeding.")

if 'HAVE_TORCH' not in globals():
    try:
        import importlib.util as _ilu
        HAVE_TORCH = _ilu.find_spec("torch") is not None
    except Exception:
        HAVE_TORCH = False

if HAVE_TORCH:
    try:
        import torch
        torch.manual_seed(0)
        if hasattr(torch, "cuda"):
            try:
                torch.cuda.manual_seed_all(0)
            except Exception:
                pass
        try:
            torch.use_deterministic_algorithms(True)
        except Exception:
            pass
        if hasattr(torch, "backends") and hasattr(torch.backends, "cudnn"):
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
        print("Torch deterministic mode enabled.")
    except Exception as e:
        print(f"Warning: could not fully enable torch determinism: {e}")

In [ ]:
# Helper: run a CLI that prints a single-line JSON and return it as dict.
import json, subprocess, sys
from typing import List, Dict

def _fallback_cmd(cmd: List[str]) -> List[str]:
    if not cmd:
        return cmd
    c0, rest = cmd[0], cmd[1:]
    if c0 == "dch-run":
        return [sys.executable, "scripts/run_experiment.py", *rest]
    if c0 == "dch-baseline-norse":
        return [sys.executable, "-m", "baselines.norse_sg", *rest]
    if c0 == "dch-baseline-bindsnet":
        return [sys.executable, "-m", "baselines.bindsnet_stdp", *rest]
    return cmd

def run_json(cmd: List[str]) -> Dict:
    try:
        proc = subprocess.run(cmd, capture_output=True, text=True)
        if proc.returncode != 0:
            # fallback to python/module invocation if console script unavailable
            fc = _fallback_cmd(cmd)
            proc = subprocess.run(fc, capture_output=True, text=True)
    except FileNotFoundError:
        fc = _fallback_cmd(cmd)
        proc = subprocess.run(fc, capture_output=True, text=True)
    except Exception as e:
        print(f"Error starting process: {e}")
        return {}

    if proc.returncode != 0:
        err = (proc.stderr or "").strip()
        out = (proc.stdout or "").strip()
        print("Command failed:", " ".join(cmd))
        if err:
            print("stderr:", err)
        if ("Missing optional dependency" in err) or ("SNN is enabled" in err):
            print("Optional dependency gate hit. Install the optional packages or disable the feature as documented.")
        return {}

    first = ""
    for ln in (proc.stdout or "").splitlines():
        if ln.strip():
            first = ln.strip()
            break
    try:
        return json.loads(first) if first else {}
    except Exception:
        print("Failed to parse JSON from output. First non-empty line:")
        print(first)
        return {}

## Quickstart overview and datasets (opt-in)

Base install and tests:

```bash
pip install -e .
pytest -q
```

DCH-only CLI examples (no torch required):

```bash
dch-run experiment=dvs_gesture snn.enabled=false
dch-run experiment=nmnist snn.enabled=false
```

Datasets are NOT auto-downloaded. To opt-in, first install tonic and then use [scripts/download_datasets.py](scripts/download_datasets.py:1):

```bash
# Optional (commented):
# pip install tonic
# python scripts/download_datasets.py --dataset dvs_gesture --root ./data
# python scripts/download_datasets.py --dataset nmnist --root ./data
```

Optional baselines require additional packages; see [docs/BASELINES.md](docs/BASELINES.md:1).

In [ ]:
# DCH-only CLI demo: DVS Gesture (torch-free)
import json as _json
res = run_json(["dch-run", "experiment=dvs_gesture", "snn.enabled=false"])
print(_json.dumps(res, indent=2, sort_keys=True))

In [ ]:
# DCH-only CLI demo: NMNIST (torch-free)
import json as _json
res = run_json(["dch-run", "experiment=nmnist", "snn.enabled=false"])
print(_json.dumps(res, indent=2, sort_keys=True))

In [ ]:
# Optional baselines (guarded): Norse and BindsNET
import json as _json
if HAVE_TORCH and HAVE_NORSE:
    print("Running Norse baseline (synthetic config)...")
    r = run_json(["dch-baseline-norse", "--config", "configs/baselines/norse_sg.yaml"])
    print(_json.dumps(r, indent=2, sort_keys=True))
else:
    print("Norse baseline is optional. Install: pip install 'torch>=2.2' 'norse>=0.0.9' (or conda equivalents).")

if HAVE_TORCH and HAVE_BINDSNET:
    print("Running BindsNET STDP baseline (synthetic config)...")
    r = run_json(["dch-baseline-bindsnet", "--config", "configs/baselines/bindsnet_stdp.yaml"])
    print(_json.dumps(r, indent=2, sort_keys=True))
else:
    print("BindsNET baseline is optional. Install: pip install 'torch>=2.2' 'bindsnet>=0.3' (or conda).")

In [ ]:
# Programmatic minimal pipeline example (torch-free)
from dch_core.interfaces import Event, make_vertex_id
from dch_pipeline.pipeline import (
    DCHPipeline,
    PipelineConfig,
    DHGConfig,
    TraversalConfig,
    PlasticityConfig,
    FSMConfig,
    SNNConfig,
)

# Construct a minimal PipelineConfig with SNN disabled
cfg = PipelineConfig(
    dhg=DHGConfig(),
    traversal=TraversalConfig(),
    plasticity=PlasticityConfig(),
    fsm=FSMConfig(),
    enable_abstraction=False,
    abstraction_params={},
    fsm_promotion_limit_per_step=FSMConfig.promotion_limit_per_step,
    snn=SNNConfig(enabled=False),
)

pipeline, encoder = DCHPipeline.from_defaults(
    cfg=cfg,
    connectivity_map={1: [0], 2: [0, 1]},
)

events = [
    Event(neuron_id=0, t=1000),
    Event(neuron_id=1, t=1400),
    Event(neuron_id=2, t=2000),
    Event(neuron_id=2, t=2600),
    Event(neuron_id=1, t=3100),
]
target = make_vertex_id(1, 3100)
metrics = pipeline.step(events=events, target_vertices=[target], sign=+1, freeze_plasticity=False)
summary = {
    "status": "ok",
    "metrics_subset": {k: (int(v) if isinstance(v, (bool, int)) else float(v) if isinstance(v, float) else v)
        for k, v in metrics.items() if k in ("n_events_ingested", "n_vertices_new", "n_candidates", "n_admitted")},
}
import json as _json
print(_json.dumps(summary, indent=2, sort_keys=True))